In [3]:
import * as E from "fp-ts/Either";
import * as A from "fp-ts/Array";
import * as F from "fp-ts/function";

import * as P from "../lib/promise";
import { request1, request2, request3, request4, request5, request6 } from '../mocks/requests'


5:20 - File '/Users/tiansivive/Workspace/fp-ts-tutorial/notebooks/lib/promise.ts' is not under 'rootDir' '/Users/tiansivive/Workspace/fp-ts-tutorial/notebooks/01-do_notation'. 'rootDir' is expected to contain all source files.
6:76 - File '/Users/tiansivive/Workspace/fp-ts-tutorial/notebooks/mocks/requests.ts' is not under 'rootDir' '/Users/tiansivive/Workspace/fp-ts-tutorial/notebooks/01-do_notation'. 'rootDir' is expected to contain all source files.


Simple sequential async requests

In [ ]:
const sequential = request1(0)
    .then(request2)
    .then(request3)
    .then(request4)
    .then(request5)
    .then(request6);

console.log("sequential:", await sequential);

Sometimes we need to keep the previous responses around, which causes nesting

In [ ]:
const nested = request1(0).then(response1 =>
    request2({ response1 }).then(response2 =>
      request3({ response1, response2 }).then(response3 =>
        request4({ response1, response2, response3 }).then(response4 =>
          request5({ response1, response2, response3, response4 })
        )
      )
    )
  );

  console.log("nested:", await nested);

Luckily, the `await`/`async` syntax helps with this

Although there are some issues: it hides the `Promise` data type away, it allows imperative style which can be tricky to follow (like awaiting within a loop or something)

Most importantly, `await`/`async` is specific to `Promises`, you cannot use it for other data types
 

In [4]:
const awaited1 = await request1(0);
const awaited2 = await request2({ awaited1 });
const awaited3 = await request3({ awaited1, awaited2 });
const awaited4 = await request4({ awaited1, awaited2, awaited3 });
const awaited5 = await request5({ awaited1, awaited2, awaited3, awaited4 });

console.log("awaited:", { awaited1, awaited2, awaited3, awaited4, awaited5 });

1:24 - Cannot find name 'request1'.
2:24 - Cannot find name 'request2'.
3:24 - Cannot find name 'request3'.
4:24 - Cannot find name 'request4'.
5:24 - Cannot find name 'request5'.


Enter `do notation`
this is equivalent to the previous `await`/`async` code

`bind` receives a key name and a function that returns a promise. The result of that function will be the value of the supplied key in an internal object
We can then access that internal object in each subsequent `bind`

In [ ]:
const piped = F.pipe(
    P.Do,
    P.bind("response1", () => request1(0)),
    P.bind("response2", ({ response1 }) => request2({ response1 })),
    P.bind("response3", ({ response1, response2 }) => request3({ response1, response2 })),
    P.bind("response4", ({ response1, response2, response3 }) =>
      request4({ response1, response2, response3 })
    ),
    P.bind("response5", ({ response1, response2, response3, response4 }) =>
      request5({ response1, response2, response3, response4 })
    )
  );

  console.log("piped", await piped);

Sometimes, when the types are right, we can achieve more concise notation

In [ ]:
const piped2 = F.pipe(
    P.Do,
    P.apS("response1", request1(0)),
    P.bind("response2", request2),
    P.bind("response3", request3),
    P.bind("response4", request4),
    P.bind("response5", request5)
  );
  
 console.log("piped", await piped2);